In [1]:
# ----------------------- 1) IMPORT LIBRARIES -----------------------
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer

from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

import tensorflow as tf

Using TensorFlow backend.
C:\Users\DespoinaK\Miniconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\DespoinaK\Miniconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\DespoinaK\Miniconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Despoina

In [2]:
# ----------------------- 2) GET THE VOCABULARY OF DATASET -----------------------

#READ .xls FILE with preprocessed data
movie_reviews = pd.read_excel("C:\\Users\\DespoinaK\\Desktop\\NN-Project\\preprocessedData.xls")

X = movie_reviews['review']
y = movie_reviews['sentiment']
y = np.array(list(map(lambda x: 1 if x=="positive" else 0, y)))

#Split dataset: 80% -->training, 20% -->testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)


# -------------------- TFIDF(VECTORIZATION) --------------------

# settings that you use for count vectorizer will go here
tfidf_vectorizer=TfidfVectorizer(use_idf=True)

# just send in all your docs here
tfidf_vectorizer_vectors_XTrain=tfidf_vectorizer.fit_transform(X_train)

tfidf_vectorizer_vectors_XTest=tfidf_vectorizer.transform(X_test)

vocabulary = tfidf_vectorizer.get_feature_names()

In [3]:
tfidf_vectorizer_vectors_XTest


<10000x75185 sparse matrix of type '<class 'numpy.float64'>'
	with 939488 stored elements in Compressed Sparse Row format>

In [4]:
# ----------------------- 3) DEFINE PREPROCESS FUNCTIONS -----------------------

def preprocess_text(sen):
    
    # -------------------- REMOVE HTML TAGS --------------------
    #remove_tags function: simply replaces anything between opening and closing <> with an empty space
    clean = re.compile('<.*?>')
    sentence = re.sub(clean, ' ', sen)

    # -------------------- REMOVE PUNCTUATIONS AND NUMBERS --------------------
    word1 = "can't" #it's a special exception because when we cut the apostrophe, then the can't --> can that we don't want it
    word2 = "won't" #it's a special exception because when we cut the apostrophe, then the won't --> won that we don't want it
    
    if word1 in sentence:
        sentence = sentence.replace(word1,"can not")
    
    if word2 in sen:
        sentence = sentence.replace(word2,"will not")
    
    sentence = re.sub('[^a-zA-Z]', ' ', sentence) #remove punctuations and numbers
    
    
    # -------------------- SINGLE CHARACTER REMOVAL --------------------
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # -------------------- REMOVE MULTIPLE SPACES --------------------
    sentence = re.sub(r'\s+', ' ', sentence)
    
    # -------------------- CONVERT TO LOWERCASE --------------------
    sentence = sentence.lower()
    
    #-----------------------------------------------------------
    negations = {    'aren':'are not', 
                     'hasn':'has not',
                     'wasn':'was not',
                  '  doesn':'does not',
                  'shouldn':'should not',
                     'didn':'did not',
                    'mustn':'must not',
                     'hadn':'had not',
                    'weren':'were not',
                     'shan':'shall not',
                    'needn':'need not',
                   'wouldn':'would not',
                      'don':'do not',
                      'ain':'is not',
                    'haven':'have not',
                      'isn':'is not',
                   'mightn':'might not',
                   'couldn':'could not' }
    
    #replace only the whole word and NOT the part of the word (e.g aren --> are not BUT arena --> arena)
    sentence = ' '.join(negations[i] if i in negations else i for i in sentence.split())
    
    # -------------------- STOP-WORDS REMOVAL --------------------
    #stop_words = set(stopwords.words('english'))
    stop_words = set(stopwords.words('english')) - set(['no','nor','any','few','not'])
    word_tokens = word_tokenize(sentence)
    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    filtered_sentence = [] 
  
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w)

    sentence = TreebankWordDetokenizer().detokenize(filtered_sentence)
    
    # -------------------- LEMMATIZATION --------------------
    #Let's trying the lemmatization approach as it is more accurate 
    # 1. Init Lemmatizer
    lemmatizer = WordNetLemmatizer()  

    sentence = word_tokenize(sentence)
    #print(sen)
    filteredSen = [] 
    for i in sentence:
        wordnetPOS = get_wordnet_pos(i)
        #print (wordnetPOS)

        # 2. Lemmatize Single Word with the appropriate POS tag
        lemma = lemmatizer.lemmatize(i,wordnetPOS)
        #print("lemma of "+i+" is: "+lemma)
        i = i.replace(i,lemma)
        filteredSen.append(i)
 
        sentence = TreebankWordDetokenizer().detokenize(filteredSen)      
        #print(sen)
    
    return sentence

def get_wordnet_pos(word):
    
    tag_dict = {   'NN':'n',
                  'NNS':'n',
                  'NNP':'n',
                 'NNPS':'n',
                   'JJ':'a',
                  'JJR':'a',
                  'JJS':'a',
                   'RB':'r',
                  'RBR':'r',
                  'RBS':'r',
                  'VB':'v',
                  'VBD':'v',
                  'VBG':'v',
                  'VBN':'v',
                  'VBP':'v',
                  'VBZ':'v'}
    
    
    wordPOS = nltk.pos_tag([word])
    #print (wordPOS[0][0],wordPOS[0][1])
    tag = wordPOS[0][1]
    
    if tag in tag_dict:  
        return tag_dict.get(tag)
    else:
        return 'n'


In [5]:
# -------------- 4) LOAD THE MODEL AGAIN, MAKE PREDICTION AND COUNT THE MEAN OF CLASSIFICATION's RUN TIME --------------
import timeit
import statistics

new_model = tf.keras.models.load_model('C:\\Users\\DespoinaK\\Desktop\\NN-Project\\my_model.h5')

#in sumA array we save the prediction runtime (sec) for each review from dataset
sumA = []

for i in range(10000):
    #start time for each row
    start = timeit.default_timer()
    
    prediction = new_model.predict(tfidf_vectorizer_vectors_XTest.tocsr()[i,:])
    prediction = prediction[0][0]
    if (prediction >= 0.50) & (prediction <= 1.0):
        print("Your review was positive (",prediction,")")
        
        #stop time for each row
        stop = timeit.default_timer()
        
        #update the sum of sec for each row
        sumA.append(stop - start)
        print('Time: ', stop - start)
        print('\n')
    else:
        print("Your review was negative(",prediction,")")
        
        #stop time for each row   
        stop = timeit.default_timer()
        
        #update the sum of sec for each row
        sumA.append(stop - start)
        print('Time: ', stop - start)
        print('\n')
print('Time(avg): ', statistics.mean(sumA))


Your review was positive ( 0.5981205 )
Time:  0.0469048543556454


Your review was positive ( 0.9741589 )
Time:  0.003927981494182831


Your review was negative( 0.01066718 )
Time:  0.0038253487943791242


Your review was positive ( 0.98912793 )
Time:  0.003478108160042126


Your review was negative( 0.03135939 )
Time:  0.00439496027829378


Your review was positive ( 0.9812259 )
Time:  0.0025481418634782926


Your review was positive ( 0.9833456 )
Time:  0.002982620292648619


Your review was negative( 0.43905392 )
Time:  0.004409214819936835


Your review was negative( 0.145251 )
Time:  0.002847487237907842


Your review was negative( 0.027755035 )
Time:  0.003562495046548264


Your review was negative( 0.04462037 )
Time:  0.003783725532791493


Your review was positive ( 0.9617036 )
Time:  0.0037711815361483048


Your review was negative( 0.0055576465 )
Time:  0.0033167467486840962


Your review was negative( 0.006512431 )
Time:  0.0033560892836064227


Your review was negative( 0.0

Your review was positive ( 0.5712941 )
Time:  0.003610960488121151


Your review was positive ( 0.96906483 )
Time:  0.004265529040207383


Your review was positive ( 0.993349 )
Time:  0.0035944252198198967


Your review was negative( 0.15159245 )
Time:  0.0033532383752792327


Your review was negative( 0.008110361 )
Time:  0.004165747248730867


Your review was positive ( 0.9866182 )
Time:  0.0037010491912852217


Your review was negative( 0.05206248 )
Time:  0.003696487737958165


Your review was positive ( 0.922646 )
Time:  0.003475257251714936


Your review was positive ( 0.9495263 )
Time:  0.003744382997865614


Your review was negative( 0.1150138 )
Time:  0.0030544631825151214


Your review was negative( 0.006823435 )
Time:  0.003440476170112561


Your review was negative( 0.16582997 )
Time:  0.0031211744373855765


Your review was positive ( 0.9845536 )
Time:  0.0032295089538472155


Your review was positive ( 0.9884785 )
Time:  0.0033087642053644117


Your review was positive ( 

Your review was positive ( 0.94809705 )
Time:  0.0037523655411852985


Your review was positive ( 0.9785593 )
Time:  0.002848627601238718


Your review was negative( 0.37269154 )
Time:  0.005447515632955202


Your review was positive ( 0.91445017 )
Time:  0.005322075666526871


Your review was negative( 0.022870997 )
Time:  0.0027859076180263287


Your review was negative( 0.0021819584 )
Time:  0.005214311331734223


Your review was negative( 0.18155563 )
Time:  0.004548909328001116


Your review was positive ( 0.98395294 )
Time:  0.003436484898454495


Your review was negative( 0.02611886 )
Time:  0.0034005634535212437


Your review was negative( 0.004497737 )
Time:  0.004520970426387549


Your review was negative( 0.2973051 )
Time:  0.0035471001415778858


Your review was positive ( 0.99288565 )
Time:  0.004115571262158113


Your review was negative( 0.12689243 )
Time:  0.004076228727235787


Your review was negative( 0.0327224 )
Time:  0.003586442676500212


Your review was positive

Your review was negative( 0.030968988 )
Time:  0.004688033654403512


Your review was negative( 0.0101214405 )
Time:  0.005107687360268898


Your review was positive ( 0.9707098 )
Time:  0.003791137894442187


Your review was positive ( 0.98023134 )
Time:  0.003417668903487936


Your review was negative( 0.0024575477 )
Time:  0.003304772933702793


Your review was positive ( 0.9938816 )
Time:  0.00373697063621492


Your review was negative( 0.116101086 )
Time:  0.0038658316926358793


Your review was negative( 0.42910847 )
Time:  0.004888167419021983


Your review was negative( 0.13885061 )
Time:  0.0043892584616394


Your review was positive ( 0.67435837 )
Time:  0.003680522651322349


Your review was negative( 0.14608079 )
Time:  0.003691926284634661


Your review was negative( 0.008559034 )
Time:  0.002675292374902938


Your review was positive ( 0.98062474 )
Time:  0.0030099890126003004


Your review was negative( 0.023538206 )
Time:  0.003236921315501462


Your review was positive

Your review was negative( 0.013254167 )
Time:  0.004217633780299934


Your review was negative( 0.09038467 )
Time:  0.005585499596023169


Your review was positive ( 0.9638182 )
Time:  0.0044109253649295965


Your review was negative( 0.05587227 )
Time:  0.004658384207793631


Your review was negative( 0.45415953 )
Time:  0.004117281807154427


Your review was positive ( 0.8750779 )
Time:  0.0036708295630063503


Your review was positive ( 0.8521495 )
Time:  0.004501584249759105


Your review was negative( 0.017104955 )
Time:  0.00306130536250393


Your review was negative( 0.014279673 )
Time:  0.0029626639343582895


Your review was negative( 0.27869 )
Time:  0.0031639380623076363


Your review was negative( 0.0071652643 )
Time:  0.00332701001866198


Your review was negative( 0.026390376 )
Time:  0.0033384136519742924


Your review was negative( 0.15469219 )
Time:  0.0043219770250999545


Your review was positive ( 0.9134113 )
Time:  0.0038190767960593064


Your review was negative( 

...

Your review was positive ( 0.7720804 )
Time:  0.0030949460807718765


Your review was positive ( 0.74828154 )
Time:  0.003414247813495308


Your review was negative( 0.10306051 )
Time:  0.003753505904519727


Your review was positive ( 0.9004455 )
Time:  0.002974637749332487


Your review was positive ( 0.9936109 )
Time:  0.0028982334061495862


Your review was positive ( 0.92777824 )
Time:  0.003130297344036137


Your review was negative( 0.105184205 )
Time:  0.003541398324919953


Your review was negative( 0.49602675 )
Time:  0.00343990598844357


Your review was positive ( 0.93200594 )
Time:  0.0035727583165297006


Your review was positive ( 0.9829525 )
Time:  0.003411396905171671


Your review was negative( 0.031459205 )
Time:  0.003686224467983834


Your review was positive ( 0.9935423 )
Time:  0.003812234616070498


Your review was positive ( 0.9938313 )
Time:  0.004047719643956782


Your review was positive ( 0.970861 )
Time:  0.004504435158089848


Your review was negative( 0.

Your review was positive ( 0.70384747 )
Time:  0.006090680551729122


Your review was negative( 0.050471764 )
Time:  0.004593953679581375


Your review was positive ( 0.9461385 )
Time:  0.003369203461915049


Your review was negative( 0.112984724 )
Time:  0.003391440546870683


Your review was negative( 0.09598584 )
Time:  0.0031639380623076363


Your review was negative( 0.17851175 )
Time:  0.0035299946916111935


Your review was negative( 0.03586417 )
Time:  0.004010087654023664


Your review was positive ( 0.9207989 )
Time:  0.003692496466300099


Your review was positive ( 0.5556502 )
Time:  0.003386308911885294


Your review was positive ( 0.9296006 )
Time:  0.003967324029105157


Your review was negative( 0.2563614 )
Time:  0.00550567416284764


Your review was negative( 0.013660215 )
Time:  0.007282360232792939


Your review was positive ( 0.989707 )
Time:  0.003394291455201426


Your review was positive ( 0.9829616 )
Time:  0.0028212588812976946


Your review was negative( 0.02

Your review was negative( 0.010522983 )
Time:  0.0028617417795544497


Your review was negative( 0.034541227 )
Time:  0.0035094681516483206


Your review was negative( 0.006276814 )
Time:  0.004538646058023232


Your review was positive ( 0.96611553 )
Time:  0.0029968748342952267


Your review was positive ( 0.9321061 )
Time:  0.003285386757070796


Your review was positive ( 0.98653746 )
Time:  0.0032323598621744054


Your review was negative( 0.027877888 )
Time:  0.003644031024720107


Your review was positive ( 0.9539246 )
Time:  0.0034592921650826725


Your review was negative( 0.4469204 )
Time:  0.003626355393095082


Your review was negative( 0.0036749018 )
Time:  0.0046082082212208775


Your review was negative( 0.049819857 )
Time:  0.004052281097280286


Your review was positive ( 0.6839333 )
Time:  0.0042792134001814475


Your review was positive ( 0.7077152 )
Time:  0.005917915507062332


Your review was positive ( 0.9700523 )
Time:  0.004872772514055157


Your review was pos

Your review was negative( 0.07639325 )
Time:  0.0039952629307222765


Your review was positive ( 0.9656236 )
Time:  0.004494171888104859


Your review was negative( 0.023474447 )
Time:  0.004615050401213239


Your review was negative( 0.120362744 )
Time:  0.003109770804073264


Your review was negative( 0.31373307 )
Time:  0.0034005634535247964


Your review was negative( 0.007295747 )
Time:  0.0035927146748235828


Your review was positive ( 0.56322706 )
Time:  0.003877235325944639


Your review was negative( 0.29545587 )
Time:  0.003282535848747159


Your review was positive ( 0.8139631 )
Time:  0.0033104747503642784


Your review was positive ( 0.7477222 )
Time:  0.006595291325766084


Your review was positive ( 0.9949738 )
Time:  0.005539314881112034


Your review was positive ( 0.9650918 )
Time:  0.0026336691133153067


Your review was positive ( 0.98931676 )
Time:  0.002669590558248558


Your review was negative( 0.47853994 )
Time:  0.0029159090377817165


Your review was negativ

Your review was negative( 0.07383072 )
Time:  0.0035961357648162107


Your review was negative( 0.019471627 )
Time:  0.006008574391884736


Your review was positive ( 0.991621 )
Time:  0.003844734971011121


Your review was negative( 0.43517494 )
Time:  0.0032238071371963883


Your review was positive ( 0.90233785 )
Time:  0.0035841619498384603


Your review was negative( 0.18230586 )
Time:  0.003998113839053019


Your review was positive ( 0.99361414 )
Time:  0.003988420750737021


Your review was positive ( 0.8024827 )
Time:  0.004168027975396171


Your review was negative( 0.04925382 )
Time:  0.004689174017734388


Your review was negative( 0.4563543 )
Time:  0.0049252292272896625


Your review was negative( 0.026615014 )
Time:  0.008211186166022344


Your review was positive ( 0.96366084 )
Time:  0.004538075876354242


Your review was positive ( 0.9935773 )
Time:  0.0031029286240880083


Your review was positive ( 0.95633423 )
Time:  0.0029159090377817165


Your review was positive